# Packages and Data Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# %matplotlib inline

#Stats and other tools
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
#from sklearn.metrics import classification_report,confusion_matrix, precision_score, recall_score, accuracy_score, f1_score
#from sklearn.preprocessing import StandardScaler, PolynomialFeatures
#import scipy.stats as stats
#from scipy.stats import uniform
#from scipy.stats import randint as sp_randint
#from sklearn.pipeline import Pipeline


#Models we will test and try
from sklearn.neural_network import MLPClassifier

#from keras import models
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers import Dense, Dropout
#from keras import regularizers
from keras import backend as K

cv_k_global = 5 #the amount of f_folds to be used in all CV

datasets=['1','2','deck_pred_1','deck_pred_2']

Using TensorFlow backend.


# DNN in Keras

In [2]:
def my_DNN_classifier(in_layer,optimizer='adam',neurons=64,dropout=0.1,activation='relu',activation_final='sigmoid',shape='one'):
    K.clear_session()
    model=None
    model = Sequential()
    model.add(Dense(neurons, input_dim=in_layer, kernel_initializer='normal',activation=activation))
    model.add(Dropout(dropout))
    if(shape=='none'):
        None
    if(shape=='one'):
        model.add(Dense(neurons, kernel_initializer='normal',activation=activation))
        model.add(Dropout(dropout))
    if(shape=='two'):
        model.add(Dense(neurons, kernel_initializer='normal',activation=activation))
        model.add(Dropout(dropout))
        model.add(Dense(neurons, kernel_initializer='normal',activation=activation))
        model.add(Dropout(dropout))
    model.add(Dense(1, kernel_initializer='normal', activation=activation_final))
    model.compile(loss='binary_crossentropy', metrics=['accuracy'],optimizer=optimizer)
    return model

In [3]:
for dataset in ['deck_pred_2']:#datasets
    print('Grid-Searching for Data-Set '+dataset)
    if('deck_pred' not in dataset):
        data=pd.read_csv('./data_train_'+dataset+'.csv',index_col='PassengerId').drop(['Missing_Embark','Missing_Deck'],axis=1)
    if('deck_pred' in dataset):
        data=pd.read_csv('./data_train_'+dataset+'.csv',index_col='PassengerId').drop(['Missing_Embark'],axis=1)
    X=data.drop('Survived',axis=1).as_matrix()
    Y=data['Survived'].as_matrix()
    K.clear_session()
    classifier = None
    classifier = KerasClassifier(build_fn=my_DNN_classifier,verbose=0,batch_size=1024,in_layer=X.shape[1])
    param_grid ={
        'epochs': [1000],
        'optimizer': ['adam','adagrad','rmsprop','adadelta','nadam'],
        'dropout' : np.linspace(0,1,11),
        'neurons' : [16,32],
        'activation' : ['relu','sigmoid','tanh'],
        #'activation_final' : ['relu','sigmoid','tanh'],
        'shape':['none','one','two']
    }
    gscv=GridSearchCV(estimator=classifier,param_grid=param_grid,scoring='accuracy',cv=cv_k_global,verbose=1,n_jobs=-1)
    gscv.fit(X,Y)
    results_df=pd.DataFrame(gscv.cv_results_)
    results_df.to_csv('./CV_DNN_'+dataset+'.csv')
    del classifier
    K.clear_session()

Grid-Searching for Data-Set deck_pred_2
Fitting 5 folds for each of 990 candidates, totalling 4950 fits


[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:   13.4s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 418 tasks      | elapsed:  4.6min
[Parallel(n_jobs=-1)]: Done 768 tasks      | elapsed:  8.8min
[Parallel(n_jobs=-1)]: Done 1218 tasks      | elapsed: 14.3min
[Parallel(n_jobs=-1)]: Done 1768 tasks      | elapsed: 20.1min
[Parallel(n_jobs=-1)]: Done 2418 tasks      | elapsed: 27.9min
[Parallel(n_jobs=-1)]: Done 3168 tasks      | elapsed: 37.1min
[Parallel(n_jobs=-1)]: Done 4018 tasks      | elapsed: 46.5min
[Parallel(n_jobs=-1)]: Done 4950 out of 4950 | elapsed: 57.7min finished


# DNN in SciKit Learn (MLPClassifier)

In [4]:
for dataset in datasets:
    print('Grid-Searching for Data-Set '+dataset)
    if('deck_pred' not in dataset):
        data=pd.read_csv('./data_train_'+dataset+'.csv',index_col='PassengerId').drop(['Missing_Embark','Missing_Deck'],axis=1)
    if('deck_pred' in dataset):
        data=pd.read_csv('./data_train_'+dataset+'.csv',index_col='PassengerId').drop(['Missing_Embark'],axis=1)
    X=data.drop('Survived',axis=1).as_matrix()
    Y=data['Survived'].as_matrix()
    MLP_class = MLPClassifier(max_iter=1000)
    param_grid ={
        'hidden_layer_sizes': [(8*x,)*y for x in range(1,11) for y in range(1,5)],
        'activation':['logistic', 'tanh', 'relu'],
        'solver':['lbfgs','adam'],
        'alpha':[10**x for x in range(-4,0)],
    }
    gscv=GridSearchCV(MLP_class,param_grid,scoring='accuracy',cv=cv_k_global,verbose=1,n_jobs=-1)
    gscv.fit(X,Y)
    results_df=pd.DataFrame(gscv.cv_results_)
    results_df.to_csv('./CV_MLP_'+dataset+'.csv')

Grid-Searching for Data-Set 1
Fitting 5 folds for each of 960 candidates, totalling 4800 fits


[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   20.5s
[Parallel(n_jobs=-1)]: Done 418 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 768 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done 1218 tasks      | elapsed:  6.5min
[Parallel(n_jobs=-1)]: Done 1768 tasks      | elapsed:  9.6min
[Parallel(n_jobs=-1)]: Done 2418 tasks      | elapsed: 16.8min
[Parallel(n_jobs=-1)]: Done 3168 tasks      | elapsed: 24.4min
[Parallel(n_jobs=-1)]: Done 4018 tasks      | elapsed: 32.1min
[Parallel(n_jobs=-1)]: Done 4800 out of 4800 | elapsed: 39.3min finished


Grid-Searching for Data-Set 2
Fitting 5 folds for each of 960 candidates, totalling 4800 fits


[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   19.1s
[Parallel(n_jobs=-1)]: Done 418 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 768 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 1218 tasks      | elapsed:  6.0min
[Parallel(n_jobs=-1)]: Done 1768 tasks      | elapsed:  8.7min
[Parallel(n_jobs=-1)]: Done 2418 tasks      | elapsed: 15.3min
[Parallel(n_jobs=-1)]: Done 3168 tasks      | elapsed: 22.1min
[Parallel(n_jobs=-1)]: Done 4018 tasks      | elapsed: 29.1min
[Parallel(n_jobs=-1)]: Done 4800 out of 4800 | elapsed: 35.9min finished


Grid-Searching for Data-Set deck_pred_1
Fitting 5 folds for each of 960 candidates, totalling 4800 fits


[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   16.6s
[Parallel(n_jobs=-1)]: Done 418 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 768 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 1218 tasks      | elapsed:  5.4min
[Parallel(n_jobs=-1)]: Done 1768 tasks      | elapsed:  7.9min
[Parallel(n_jobs=-1)]: Done 2418 tasks      | elapsed: 14.1min
[Parallel(n_jobs=-1)]: Done 3168 tasks      | elapsed: 20.4min
[Parallel(n_jobs=-1)]: Done 4018 tasks      | elapsed: 27.1min
[Parallel(n_jobs=-1)]: Done 4800 out of 4800 | elapsed: 33.5min finished


Grid-Searching for Data-Set deck_pred_2
Fitting 5 folds for each of 960 candidates, totalling 4800 fits


[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   18.5s
[Parallel(n_jobs=-1)]: Done 418 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 768 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 1218 tasks      | elapsed:  6.0min
[Parallel(n_jobs=-1)]: Done 1768 tasks      | elapsed:  8.7min
[Parallel(n_jobs=-1)]: Done 2418 tasks      | elapsed: 15.4min
[Parallel(n_jobs=-1)]: Done 3168 tasks      | elapsed: 22.2min
[Parallel(n_jobs=-1)]: Done 4018 tasks      | elapsed: 29.2min
[Parallel(n_jobs=-1)]: Done 4800 out of 4800 | elapsed: 36.0min finished
